In [32]:
import psycopg2

In [33]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=alpha")
except psycopg2.Error as e:
    print('Could not make connection to the Postgre database')
    print(e)

In [34]:
try:
    cursor = conn.cursor()
except psycopg2.Error as e:
    print('Could not create a cursor to the database')
    print(e)

In [35]:
conn.set_session(autocommit=True)

<h3>Starting at the 3NF Form</h3>

In [36]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ALBUM_LIBRARY_3NF (
            ALBUM_ID INT,
            ALBUM_NAME VARCHAR,
            ARTIST_ID INT,
            YEAR INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create album table')
    print(e)

In [37]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SONG_LIBRARY_3NF (
            SONG_ID INT,
            SONG_NAME VARCHAR,
            ALBUM_ID INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song table')
    print(e)

In [38]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ARTIST_LIBRARY_3NF (
            ARTIST_ID INT,
            ARTIST_NAME VARCHAR
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song table')
    print(e)

In [39]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SONG_LENGTH_3NF (
            SONG_ID INT,
            SONG_LENGTH INT
        );
    ''')
except psycopg2.Error as e:
    print('Could not create song_length table')
    print(e)

In [40]:
query = '''
    INSERT INTO ALBUM_LIBRARY_3NF (
        ALBUM_ID,
        ALBUM_NAME,
        ARTIST_ID,
        YEAR
    ) 
    VALUES (
        %s, %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Rubber Soul', 1, 1965))
    cursor.execute(query, (2, 'Let It Be', 1, 1970))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [41]:
query = '''
    INSERT INTO SONG_LIBRARY_3NF (
        SONG_ID,
        SONG_NAME,
        ALBUM_ID
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Michele', 1))
    cursor.execute(query, (2, 'Think For Yourself', 1))
    cursor.execute(query, (3, 'In My Life', 1))
    cursor.execute(query, (4, 'Let It Be', 2))
    cursor.execute(query, (5, 'Across The Universe', 2))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [42]:
query = '''
    INSERT INTO ARTIST_LIBRARY_3NF (
        ARTIST_ID,
        ARTIST_NAME
    ) 
    VALUES (
        %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'The Beatles'))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [43]:
query = '''
    INSERT INTO SONG_LENGTH_3NF (
        SONG_ID,
        SONG_LENGTH
    ) 
    VALUES (
        %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 163))
    cursor.execute(query, (2, 137))
    cursor.execute(query, (3, 145))
    cursor.execute(query, (4, 240))
    cursor.execute(query, (5, 227))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [45]:
query = '''
    SELECT 
          A.SONG_NAME
        , B.*
        , C.ARTIST_NAME
        , D.SONG_LENGTH
    FROM 
        SONG_LIBRARY_3NF AS A 
            LEFT JOIN 
                ALBUM_LIBRARY_3NF AS B ON A.ALBUM_ID = B.ALBUM_ID
            LEFT JOIN 
                ARTIST_LIBRARY_3NF AS C ON B.ARTIST_ID = C.ARTIST_ID
            LEFT JOIN
                SONG_LENGTH_3NF AS D ON A.SONG_ID = D.SONG_ID

'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

('Michele', 1, 'Rubber Soul', 1, 1965, 'The Beatles', 163)
('Think For Yourself', 1, 'Rubber Soul', 1, 1965, 'The Beatles', 137)
('In My Life', 1, 'Rubber Soul', 1, 1965, 'The Beatles', 145)
('Let It Be', 2, 'Let It Be', 1, 1970, 'The Beatles', 240)
('Across The Universe', 2, 'Let It Be', 1, 1970, 'The Beatles', 227)


In [31]:
# Dropping all the tables
query = '''
    DROP TABLE IF EXISTS SONG_LIBRARY_3NF;
    DROP TABLE IF EXISTS ALBUM_LIBRARY_3NF;
    DROP TABLE IF EXISTS ARTIST_LIBRARY_3NF;
    DROP TABLE IF EXISTS SONG_LENGTH_3NF;
'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

<h3>Considering our query is: </h3>
<h4>SELECT ALBUM_ID, ALBUM_NAME, SUM(SONG_LENGTH) AS ALBUM_LENGTH FROM TABLE GROUP BY (ALBUM_ID)</h4>

In [47]:
# Creating a dummy table which is a central denormalised table reducing joins - shortcut to condense tables
query = '''
    SELECT 
          A.SONG_NAME
        , B.*
        , C.ARTIST_NAME
        , D.SONG_LENGTH
    INTO FLAT_TABLE
    FROM 
        SONG_LIBRARY_3NF AS A 
            LEFT JOIN 
                ALBUM_LIBRARY_3NF AS B ON A.ALBUM_ID = B.ALBUM_ID
            LEFT JOIN 
                ARTIST_LIBRARY_3NF AS C ON B.ARTIST_ID = C.ARTIST_ID
            LEFT JOIN
                SONG_LENGTH_3NF AS D ON A.SONG_ID = D.SONG_ID

'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [50]:
query = '''
    SELECT 
          ALBUM_ID
        , ALBUM_NAME
        , SUM(SONG_LENGTH) AS ALBUM_LENGTH_SEC
    FROM 
        FLAT_TABLE
    GROUP BY 
          ALBUM_ID
        , ALBUM_NAME
'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(2, 'Let It Be', 467)
(1, 'Rubber Soul', 445)
